This notebook is used to generate connectivity and gradient used for the paper.

## Code Setup

In [1]:
# If True, assumes everything is running locally.
IS_LOCAL = False

# Path to main directory
REMOTE_PATH = "/content/gdrive/Shareddrives/Birds and CS/Data/CA-paper"
LOCAL_PATH = "/Users/luca/Library/CloudStorage/GoogleDrive-luca@ucsc.edu/Shared drives/Birds and CS/Data/CA-paper"
DATA_PATH = LOCAL_PATH if IS_LOCAL else REMOTE_PATH

# Path to scratch directory where we can put a copy of the database.
REMOTE_SCRATCH_PATH = "/content/temp"
LOCAL_SCRATCH_PATH = "/Users/luca/temp/ebird"
SCRATCH_PATH = LOCAL_SCRATCH_PATH if IS_LOCAL else REMOTE_SCRATCH_PATH

In [2]:
import os
import sys

In [3]:
# Installs required packages
if not IS_LOCAL:
    !pip install git+https://github.com/ecoscape-earth/ecoscape-connectivity.git
    !pip install git+https://github.com/ecoscape-earth/ecoscape-utils.git

  Cloning https://github.com/ecoscape-earth/ecoscape-connectivity.git to /tmp/pip-req-build-byfucbl4
  Running command git clone --filter=blob:none --quiet https://github.com/ecoscape-earth/ecoscape-connectivity.git /tmp/pip-req-build-byfucbl4
  Resolved https://github.com/ecoscape-earth/ecoscape-connectivity.git to commit 35268674abddd15d892d1033fc12ec6d9a0bfd55
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 84.4 MB/s eta 0:00:00
  Created wheel for ecoscape-connectivity: filename=ecoscape_connectivity-0.0.1-py3-none-any.whl size=10990 sha256=b4c86f9acb8e42df3f717e6c0fc9a059b60f3952a98ace055e6bec1e89bdb57d
  Stored in directory: /tmp/pip-ephem-wheel-cache-i7g1i6g8/wheels/72/73/2c/f3b584a638e8a6a37479ac7d8823cb5a88a277b40f8c2732f4
Successfully built ecoscape-connectivity
  Cloning https://github.com

In [4]:
# Connecting to Drive.
if not IS_LOCAL:
    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [5]:
import time
from ecoscape_utilities import BirdRun
from ecoscape_connectivity import compute_connectivity
from ecoscape_connectivity.util import read_transmission_csv

## Bird Run Definition

In [6]:
bird_run = BirdRun(DATA_PATH)

birds = []

NUM_SIMULATIONS = 400
TRANSMISSION_EXPONENT = 1
BIRD_RUN = "Aug24-timing"

birds.append(bird_run.get_bird_run(
    "acowoo", "Acorn Woodpecker",
    run_name=BIRD_RUN,
    do_validation=False, do_gradient=False,
    hop_distance=2, num_spreads=20,
    num_simulations=NUM_SIMULATIONS,
    transmission_exponent=TRANSMISSION_EXPONENT))

birds.append(bird_run.get_bird_run(
    "acowoo", "Acorn Woodpecker",
    run_name=BIRD_RUN,
    do_validation=False, do_gradient=True,
    hop_distance=2, num_spreads=20,
    num_simulations=NUM_SIMULATIONS,
    transmission_exponent=TRANSMISSION_EXPONENT))

birds.append(bird_run.get_bird_run(
    "stejay", "Steller's Jay",
    run_name=BIRD_RUN,
    do_validation=False, do_gradient=False,
    hop_distance=2, num_spreads=6,
    num_simulations=NUM_SIMULATIONS,
    transmission_exponent=TRANSMISSION_EXPONENT))

birds.append(bird_run.get_bird_run(
    "stejay", "Steller's Jay",
    run_name=BIRD_RUN,
    do_validation=False, do_gradient=True,
    hop_distance=2, num_spreads=6,
    num_simulations=NUM_SIMULATIONS,
    transmission_exponent=TRANSMISSION_EXPONENT))

birds.append(bird_run.get_bird_run(
    "oaktit308", "Oak Titmouse",
    run_name=BIRD_RUN,
    do_validation=False, do_gradient=False,
    hop_distance=2, num_spreads=15,
    num_simulations=NUM_SIMULATIONS,
    transmission_exponent=TRANSMISSION_EXPONENT))

birds.append(bird_run.get_bird_run(
    "oaktit308", "Oak Titmouse",
    run_name=BIRD_RUN,
    do_validation=False, do_gradient=True,
    hop_distance=2, num_spreads=15,
    num_simulations=NUM_SIMULATIONS,
    transmission_exponent=TRANSMISSION_EXPONENT))


In [7]:
# Let's do the bird runs.
t = time.time()
for bird in birds:

    # Creates output folder, if missing.
    bird_run.createdir_for_file(bird.repopulation_fn)
    bird_run.createdir_for_file(bird.gradient_fn)

    transmission_d = read_transmission_csv(bird.transmission_fn)

    compute_connectivity(
        habitat_fn=bird.habitat_fn,
        terrain_fn=bird.terrain_fn,
        connectivity_fn=bird.repopulation_fn,
        flow_fn=bird.gradient_fn if bird.do_gradient else None,
        permeability_dict=transmission_d,
        gap_crossing=bird.hop_distance,
        num_gaps=bird.num_spreads,
        num_simulations=bird.num_simulations,
        seed_density=4,
        single_tile=False,
        tile_size=2048,
        tile_border=256,
    )

    print("Processed", bird.name, "gradient", bird.do_gradient, "hop", bird.hop_distance, "num", bird.num_spreads, "sim", bird.num_simulations)
    print("Time:", time.time() - t)
    t = time.time()

Processed Acorn Woodpecker gradient False hop 2 num 20 sim 400
Time: 35.97592234611511


Processed Acorn Woodpecker gradient True hop 2 num 20 sim 400
Time: 59.27016353607178


Processed Steller's Jay gradient False hop 2 num 6 sim 400
Time: 13.941468000411987


Processed Steller's Jay gradient True hop 2 num 6 sim 400
Time: 21.052608489990234


Processed Oak Titmouse gradient False hop 2 num 15 sim 400
Time: 18.7767014503479


Processed Oak Titmouse gradient True hop 2 num 15 sim 400
Time: 38.35686492919922
